## Sinhala Chatbot RAG Interface

The following is the notebook containing the final version of sinhala RAG chatbot who's knowledge corpus is based around the constitution of sri lanka.

All codes involving generating text chunks, embeddings and indices for this version are in the shared google folder: https://drive.google.com/drive/folders/1DxqovC4pKamiKBSokdmFGjJqvJ8HtDVK?usp=drive_link

This chatbot is hosted on huggingface at: https://huggingface.co/spaces/AtleeBugs/ConstitutionRAG

This chatbot is available as a react native android APK at: https://github.com/dinushasandamali/Rag_Project

## RAG Interface Codebase

Mounting drive to retrieve stored chunks and embeddings

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing faiss-cpu for nearest neighbour search and sentence transformers for creating embeddings

In [ ]:
!pip install faiss-cpu
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 50.5 MB/s eta 0:00:00


Making necessary imports

In [ ]:
import os
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import faiss
import numpy as np

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Setting directory (change this to the directory of the shortcut for the shared directory)

In [ ]:
base_dir = '/content/drive/MyDrive/LLM_Tasks/ChatBot/RAG Interface/Assets'
output_dir = f'{base_dir}/output_chunks'   # Directory where chunks will be saved

RAG interface class

In [17]:
class RAGInterFace:
    def __init__(self, transformer, chunk_dir, faiss_dir, retrieval_func):
        self.transformer = transformer
        self.retrieval_func = retrieval_func
        self.chunks = self.load_chunks(chunk_dir)
        self.faiss = self.load_faiss(faiss_dir)

    def query(self, query):
        embedding = self.embed_query([query])   # Embed query
        chunk_ids = self.get_nearest_neighbours(embedding, 200)   # Find nearest neighbours
        chunks = self.get_chunks(chunk_ids)   # Get context chunks
        formatted_query = self.format_query(query, chunks)    # Format query (with context)
        return self.retrieve_response(formatted_query)   # Retrieve response

    def load_chunks(self, chunk_dir):
        """Load all chunks from files in the specified directory."""
        chunks = []

        for filename in sorted(os.listdir(chunk_dir)):
            with open(f"{chunk_dir}/{filename}", 'r', encoding='utf-8') as file:
                chunks.append(file.read())

        return chunks

    def load_faiss(self, faiss_dir):
        return faiss.read_index(faiss_dir)

    def embed_query(self, query):
        return self.transformer.encode(query)

    def get_nearest_neighbours(self, embedding, radius, limit_k = 10):
        D, I = self.faiss.search(embedding, limit_k)
        filtered_indices = []
        for distance, idx in zip(D[0], I[0]):
            if distance <= radius:
                filtered_indices.append(idx)
            if len(filtered_indices) >= limit_k:
                break

        return filtered_indices

    def get_chunks(self, chunk_ids, soft_limit=600):
        total_words = 0
        result_chunks = []

        for i in chunk_ids:
            chunk_word_count = len(self.chunks[i].split())
            result_chunks.append(self.chunks[i])
            total_words += chunk_word_count

            if total_words >= soft_limit:
                break

        return result_chunks

    def format_query(self, query, chunks):
        context_string = "\n".join(chunks)
        return f"If relevent, Refer to the following context extracted from constitution of sri lanka:\n{context_string}\nand answer the following question in sinhala: {query}"

    def retrieve_response(self, query):
        print(query)
        return self.retrieval_func(query)



Installing gradio client (to access a hosted llama model that was previously used, but did not outperform the model which was finally chosen)

In [ ]:
!pip install gradio_client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.9 MB/s eta 0:00:00


Accessing the llama model previously used

In [ ]:
from gradio_client import Client

def retrieve_function(query):
    # client = Client("orionai/llama-3.1-70b-demo")
    # result = client.predict(
    #     user_message="answer in sinhala: can you speak sinhala?",
    #     api_name="/predict"
    # )

    # return result
    return "Done!"

Importing the sinhala sentence transformer

This was chosen off of a review done in regards to sinhala NLP:

[BERTifying Sinhala - A Comprehensive Analysis of Pre-trained Language Models for Sinhala Text Classification](https://aclanthology.org/2022.lrec-1.803) (Dhananjaya et al., LREC 2022)

In [ ]:
transformer = SentenceTransformer('stsb-xlm-r-multilingual')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Setting up RAG interface with llama model's retrieve function

In [ ]:
chunk_dir = f'{base_dir}/output_chunks'
faiss_dir = f'{base_dir}/index_file.faiss'

rag_interface = RAGInterFace(
    transformer=transformer,
    chunk_dir=chunk_dir,
    faiss_dir=faiss_dir,
    retrieval_func=retrieve_function
)

Testing the llama model's capabilities for generating sinhala responses

In [ ]:
result = rag_interface.query("මූලික මානව හිමිකම් මොනවාද?")
print(result)

If relevent, Refer to the context:
රාජ්‍ය ප්‍රතිපත්තිය මෙහෙය වීමේ මූලධර්ම සහ මූලික යුතුකම්: අයිතිවාසිකම් සහ නන් වැදෑරුම් නිදහස ක්‍රියාත්මකවීමත්, භුක්ති විඳීමත් යුතුකම් හා බැඳීම් ඉටුකිරීමෙන් වෙන්ව පැවතිය නොහැක්කේය.
බුද්ධාගම: (1) 13 වන ව්‍යවස්ථාවේ (5) වන සහ (6) වන අනුව්‍යවස්ථාවලින් ප්‍රකාශ කොට පිළිගෙන ඇති මූලික අයිතිවාසිකම් භුක්ති විඳිය හැක්කේ ද, ක්‍රියාත්මක විය හැක්කේ ද, රාජ්‍ය ආරක්ෂාව තහවුරු කිරීම පිණිස නීතියෙන් නියම කරනු ලැබිය හැකි සීමා කිරීම්වලට පමණක් යටත්ව ය.
පොදු විධිවිධාන: 12) මේ ව්‍යවස්ථාවේ “ප්‍රජා අයිතිවාසිකම්‌” යන්නෙන්‌“ අ) විදේශ ගමන්‌ බලපත්‍රයක්‌ ලබා ගැනීමේ අයිතිවාසිකම; ආ) රජයේ යම්‌ විභාගයකට පෙනී සිටීමේ අයිතිවාසිකම; ඇ) යම්‌ නිශ්චල දේපළක්‌ අයිතිව සිටීමේ අයිතිවාසිකම; ඈ) යම්‌ ලිඛිත නීතියක්‌ මගින්‌ හෝ යටතේ හභෝ වූ බලපත්‍රයක්‌, ලියාපදිංචි කිරීමක්‌ නැතහොත්‌ වෙනත්‌ බලය දීමක්‌ අවශ්‍ය වන යම්‌ රැකියාවක හෝ වෘත්තියක යෙදීමේ අයිතිවාසිකම අදහස්‌ වේ.] 158.
බුද්ධාගම: (2) 14 වන ව්‍යවස්ථාවේ (1) වන අනුව්‍යවස්ථාවේ (අ) ඡේදයෙන් ප්‍රකාශ කොට පිළිගෙන ඇති මූලික අයිතිවාසිකම භුක්ති විඳිය හැක්කේ ද, ක්‍රියාත්

Installing OpenAI in order to use their API

In [ ]:
!pip install openai

Test codes, testing the openai module

In [ ]:
# from openai import OpenAI

# client = OpenAI(
#     # This is the default and can be omitted
#     api_key=userdata.get('OAI_API_KEY'),
# )

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "respond in sinhala: what is sri lanka?",
#         }
#     ],
#     model="gpt-4o-mini",
# )

# print(chat_completion)

In [ ]:
# print(chat_completion.choices[0].message.content)

Setting up openAI retrieval function

In [ ]:
from openai import OpenAI
from google.colab import userdata

def GPT4_retrieve_function(query):
    client = OpenAI(
        api_key=userdata.get('OAI_API_KEY'),
    )

    try:
        chat_completion = client.chat.completions.create(
        messages=[
                {
                    "role": "user",
                    "content": query,
                }
            ],
            model="gpt-4o-mini",
        )

        return chat_completion.choices[0].message.content

    except Exception as e:
        return f"Error has occured: most likely the subscription for gpt-4o-mini used for this chatbot has expired"


Setting up the RAG interface utilizing GPT-4o-mini

In [19]:
rag_interface = RAGInterFace(
    transformer=transformer,
    chunk_dir=chunk_dir,
    faiss_dir=faiss_dir,
    retrieval_func=GPT4_retrieve_function
)

Testing the model

In [20]:
result = rag_interface.query("වධහිංසා පැමිණවීම ගැන ආණ්ඩුක්‍රම ව්‍යවස්ථාවේ පවසන්නේ කුමක්ද?")
print("\n\n" + result)

If relevent, Refer to the following context extracted from constitution of sri lanka:
අදාළ පිටු අග ඇති සටහන් මගින් ආණ්ඩුක්‍රම ව්‍යවස්ථාව සංශෝධනය කළ ඒ ඒ සංශෝධන දැක් වේ.
අන්ත ර්කාලීන විධිවිධාන: පාර්ලිමේන්තුව විසින්‌ අන්‍යාකාර විධිවිධාන සලස්වන්නේ නම්‌ මිස,~~~~ 1) ජාතික රාජ්‍ය සභාවේ 1973 අංක 44 දරන යුක්තිය පසිඳලීමේ පනතේ යම්‌ විධිවිධාන ආණ්ඩුක්‍රම ව්‍යවස්ථාවේ විධිවිධානවලට අනනුකූල වන්නේ ද, ඒ විධිවිධාන ඒ අනනුකූලතාවේ ප්‍රමාණයට පරිච්ඡින්න කරන ලද සේ සැලකිය යුත්තේ ය; 2) ජාතික රාජ්‍ය සභාවේ 1973 අංක 44 දරන යුක්තිය පසිඳලිමේ පනතින්‌ පිහිටුවන ලද ශ්‍රේෂ්ඨාධිකරණයේ පැවැත්ම ආණ්ඩුක්‍රම වාවස්ථාව ක්‍රියාත්මක වීම ආරම්භ වූ විට නතර වන්නේ ය.
: 173 මහජන ආරක්ෂාව 155. (1) ආණ්ඩුක්‍රම ව්‍යවස්ථාව ක්‍රියාත්මක වීම ආරම්භ වන තෙක් බලපැවැත් සංශෝධිත මහජන ආරක්ෂක ආඥාපනත පාර්ලිමේන්තුව විසින් පනවන ලද පනතක් සේ සැලකෙන්නේ ය.
අර්ථ නිරූපණය: “නීතිය” යන්නෙන්‌, පාර්ලිමේන්තුවේ පනතක්‌ සහ ආණ්ඩුක්‍රම වාවස්ථාව ක්‍රියාත්මක වීම ආරම්භ වීමට පෙර කවර අවස්ථාවක වුව ද යම්කිසි වාවස්ථාදායකයක්‌ විසින්‌ පනවන ලද පනතක්‌ අදහස්‌ වන අතර, රාජසභා ආඥාවක්‌ ද ඊට ඇත